## Monad

In Haskell, a **monad** is a design pattern used to handle computations in a flexible and modular way. It allows for chaining operations while managing side effects, such as state, I/O, or exceptions, in a pure functional manner. 

### Key Components of Monads

1. **Type Constructor**: A monad is a type constructor that takes a type and returns a new type. For example, `Maybe` is a monad that can represent a value that may or may not exist.

2. **`return` Function**: This function takes a value and puts it into a monadic context. For example, `return 5` in the `Maybe` monad would give `Just 5`.

3. **`>>=` (bind) Operator**: This operator takes a monadic value and a function that returns a monadic value, chaining them together. It allows you to apply a function to the value inside the monad while preserving the monadic context.

#### Monad Type Class

In Haskell, monads are defined by the `Monad` type class, which has the following key methods:

- `return :: a -> m a`  
  Puts a value into a monadic context.

- `(>>=) :: m a -> (a -> m b) -> m b`  
  Chains operations on monadic values.

#### Example: Maybe Monad

The `Maybe` monad is a common example used to handle computations that might fail:

```haskell
safeDivide :: Int -> Int -> Maybe Int
safeDivide _ 0 = Nothing
safeDivide x y = Just (x `div` y)

result :: Maybe Int
result = return 10 >>= \x ->
         safeDivide x 2 >>= \y ->
         safeDivide y 0  -- This will produce Nothing
```

#### Benefits of Monads

- **Chaining Operations**: Monads allow for clean and readable chaining of operations without having to explicitly handle intermediate states or effects.
  
- **Abstracting Side Effects**: They provide a way to abstract away side effects, making functions easier to reason about.

#### Common Monads in Haskell

1. **Maybe**: Handles optional values.
2. **List**: Represents non-deterministic computations.
3. **IO**: Manages input/output operations.
4. **State**: Encapsulates stateful computations.

#### Conclusion

Monads can be a challenging concept to grasp initially, but they are powerful tools in Haskell for managing side effects and structuring programs in a clear and maintainable way. Understanding monads is key to mastering functional programming in Haskell.

### The Monad Laws

1. **Left Identity**:
   If you take a value and wrap it in a monad, and then bind it to a function, it is the same as just applying the function to the value directly.

   Formally:
   ```haskell
   return a >>= f  ≡  f a
   ```

2. **Right Identity**:
   If you have a monadic value and you bind it to the `return` function, the result should be the same as the original monadic value.

   Formally:
   ```haskell
   m >>= return  ≡  m
   ```

3. **Associativity**:
   The way you nest monadic operations should not matter. If you bind a monadic value to two functions in sequence, it should be the same regardless of how you group the operations.

   Formally:
   ```haskell
   (m >>= f) >>= g  ≡  m >>= (\x -> f x >>= g)
   ```

#### Summary

These laws ensure that monadic operations behave in a predictable manner, which is essential for reasoning about code that uses monads. They guarantee that monads can be composed and that chaining operations will produce consistent results.

#### Practical Example

Consider the `Maybe` monad, which is used for computations that might fail:

- **Left Identity**: `return 5 >>= safeDivide` is equivalent to `safeDivide 5`.
- **Right Identity**: `Just 5 >>= return` results in `Just 5`.
- **Associativity**: Chaining operations with `>>=` will yield the same result regardless of how you group the operations.

### Exercise 1

Write bind in terms of fmap and join. Fear is the mind-killer, friend. You can do it.

```haskell
-- keep in mind this is (>>=) flipped
bind :: Monad m => (a -> m b) -> m a -> m b
bind = undefined
```

In [28]:
import Control.Monad (join)

bind :: Monad m => (a -> m b) -> m a -> m b
bind f ma = join $ fmap f ma

Line 4: Use =<<
Found:
join $ fmap f ma
Why not:
f =<< ma

### `do` Syntax

In Haskell, the `do` notation provides syntactic sugar for working with monads, making code more readable and easier to write. This notation simplifies the chaining of monadic operations, allowing you to write code that looks imperative while still being functional.

### Basics of `do` Notation

When you use `do` notation, you can write sequential monadic operations without explicitly using the `>>=` (bind) operator. Here’s how it works:

1. **Sequential Operations**: Each line in a `do` block corresponds to a monadic operation.
2. **Binding**: The `<-` operator is used to extract values from monads.

#### Example of `do` Notation vs. Bind Operator

Let's compare a simple example using both `do` notation and the bind operator.

#### Using Bind Operator

```haskell
import Data.Maybe

safeDivide :: Int -> Int -> Maybe Int
safeDivide _ 0 = Nothing
safeDivide x y = Just (x `div` y)

result :: Maybe Int
result = Just 10 >>= \x ->
         safeDivide x 2 >>= \y ->
         safeDivide y 0
```

#### Using `do` Notation

```haskell
import Data.Maybe

safeDivide :: Int -> Int -> Maybe Int
safeDivide _ 0 = Nothing
safeDivide x y = Just (x `div` y)

result :: Maybe Int
result = do
    x <- Just 10
    y <- safeDivide x 2
    safeDivide y 0
```

#### Explanation

- **Extraction**: In the `do` notation, `x <- Just 10` extracts the value from the `Just` constructor, similar to how `Just 10 >>= \x -> ...` works.
- **Readability**: The `do` block reads more like imperative code, making it easier to follow the flow of operations.

#### Nested Monads

You can also use `do` notation for nested monads (like `Maybe` within `List`), but you'll need to use additional operators or techniques to handle them appropriately.

#### Concise rules

```haskell
do
    EXPRESSION_1
    EXPRESSION_2
    .
    .
    .
    EXPRESSION_n
```

is equivalent to:

```haskell
EXPRESSION_n
```

and

```haskell

do
    x <- m
    SEQUENCE(x)
```

is equivalent to:

```haskell
m >>= \x -> SEQUENCE(x)
```


a `SEQUENCE` is a defined like this:
`SEQUENCE = DO_EXPRESSION | EXPRESSION`

### An Example

The following two functions `main` and `main'` are semantically the same.

In [ ]:
safeDivide :: (Fractional a, Eq a) => a -> a -> Maybe a
safeDivide x y =
    if y == 0
        then Nothing
        else Just (x / y)

main :: IO ()
main = do
  str1 <- getLine
  str2 <- getLine
  str3 <- getLine

  let num1 = read str1
  let num2 = read str2
  let num3 = read str3
  
  let x = safeDivide num1 num2 >>= safeDivide num3
  print x


main' :: IO ()
main' =
  getLine >>= \str1 ->
  getLine >>= \str2 ->
  getLine >>= \str3 ->
    (\num1 ->
      \num2 ->
        \num3 ->
          print $ safeDivide num1 num2 >>= safeDivide num3) (read str1) (read str2) (read str3)

Line 26: Collapse lambdas
Found:
\ num1
  -> \ num2
       -> \ num3 -> print $ safeDivide num1 num2 >>= safeDivide num3
Why not:
\ num1 num2 num3
  -> print $ safeDivide num1 num2 >>= safeDivide num3

### Short Exercise: Either Monad

Implement the Either Monad.

```haskell
data Sum a b =
    First a
  | Second b
  deriving (Eq, Show)

instance Functor (Sum a) where
    fmap = undefined

instance Applicative (Sum a) where
    pure = undefined
    (<*>) = undefined

instance Monad (Sum a) where
    return = pure
    (>>=) = undefined
```

In [ ]:
data Sum a b =
    First a
  | Second b
  deriving (Eq, Show)

instance Functor (Sum a) where
    fmap _ (First x) = First x
    fmap f (Second y) = Second (f y)

instance Applicative (Sum a) where
    pure = Second
    (<*>) (First x) _ = First x
    (<*>) _ (First x) = First x
    (<*>) (Second f) (Second y) = Second $ f y 

instance Monad (Sum a) where
    return = pure
    (>>=) (First x) _ = First x
    (>>=) (Second y) f = f y